<a href="https://colab.research.google.com/github/nitingarg2015/EVA8/blob/master/Assignment%20S3b/Assignment_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# use dataloader to create training and test datasets with define batch size

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm

#train function for training
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

#test function for testing outcomes
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# CNN model definition
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),                      #input: 1*28*28, output: 16*28*28, RF: 3*3
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),                     #input: 16*28*28, output: 32*28*28, RF: 5*5 
            nn.BatchNorm2d(32),
            nn.Dropout(0.05)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 8, 1),                                 #input: 32*28*28, output: 8*28*28, RF: 7*7 
            nn.MaxPool2d(2, 2),                                  #input: 8*28*28, output: 8*14*14, RF: 14*14
            nn.Dropout(0.05)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),                      #input: 8*14*14, output: 16*14*14, RF: 16*16
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),                     #input: 16*14*14, output: 16*14*14, RF: 18*18
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),                                  #input: 16*14*14, output: 16*7*7, RF: 36*36
            nn.Dropout(0.05)
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),                     #input: 16*7*7, output: 16*7*7, RF: 38*38
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),                     #input: 16*7*7, output: 16*7*7, RF: 40*40
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),                                  #input: 16*7*7, output: 16*3*3, RF: 80*80
            nn.Dropout(0.05)
        )

        self.conv8 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),                     #input: 16*3*3, output: 16*3*3, RF: 82*82
            nn.BatchNorm2d(16),
            nn.AvgPool2d(2, 2),                                  #input: 16*3*3, output: 16*1*1, RF: 164*164
            nn.Dropout(0.10)
        )

        self.fc = nn.Sequential(
            (nn.Linear(16, 10))
        )
                
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = x.view(x.size(0), -1)

        x = self.fc(x)
        
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
#instantiate Net class and check model summary

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           4,640
       BatchNorm2d-5           [-1, 32, 28, 28]              64
           Dropout-6           [-1, 32, 28, 28]               0
            Conv2d-7            [-1, 8, 28, 28]             264
         MaxPool2d-8            [-1, 8, 14, 14]               0
           Dropout-9            [-1, 8, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           1,168
      BatchNorm2d-11           [-1, 16, 14, 14]              32
          Dropout-12           [-1, 16, 14, 14]               0
           Conv2d-13           [-1, 16, 14, 14]           2,320
      BatchNorm2d-14           [-1, 16,

In [ ]:
# Set for training on GPU
#define optimizer
#set the loop for training in epochs

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.19803179800510406 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.08it/s]



Test set: Average loss: 0.0641, Accuracy: 9808/10000 (98.1%)



loss=0.07573869079351425 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.09it/s]



Test set: Average loss: 0.0397, Accuracy: 9878/10000 (98.8%)



loss=0.1322895884513855 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.55it/s]


...


Test set: Average loss: 0.0184, Accuracy: 9949/10000 (99.5%)



loss=0.045729491859674454 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.95it/s]



Test set: Average loss: 0.0209, Accuracy: 9943/10000 (99.4%)



loss=0.018251623958349228 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.25it/s]



Test set: Average loss: 0.0171, Accuracy: 9941/10000 (99.4%)

